In [21]:
import numpy as np
import pandas as pd
import json
import os
import math

%config IPCompleter.greedy=True

In [34]:
data_path = '../data'
os.chdir(data_path)
output_shape = (3, 4, 2, 9)  # width × height × color × rotation (last == any)

hard = 'Kneel Before Me/Normal.json'
extreme_plus = '[A]ddiction/ExpertPlus.json'
with open(hard) as json_data:
    data = json.load(json_data)

print(data.keys())
data['_notes']

dict_keys(['_version', '_beatsPerMinute', '_beatsPerBar', '_noteJumpSpeed', '_shuffle', '_shufflePeriod', '_events', '_notes', '_obstacles'])


[{'_time': 12,
  '_lineIndex': 3,
  '_lineLayer': 0,
  '_type': 1,
  '_cutDirection': 1},
 {'_time': 19,
  '_lineIndex': 2,
  '_lineLayer': 0,
  '_type': 1,
  '_cutDirection': 0},
 {'_time': 20,
  '_lineIndex': 2,
  '_lineLayer': 0,
  '_type': 1,
  '_cutDirection': 1},
 {'_time': 27,
  '_lineIndex': 1,
  '_lineLayer': 0,
  '_type': 0,
  '_cutDirection': 1},
 {'_time': 28,
  '_lineIndex': 1,
  '_lineLayer': 0,
  '_type': 0,
  '_cutDirection': 0},
 {'_time': 35,
  '_lineIndex': 2,
  '_lineLayer': 0,
  '_type': 1,
  '_cutDirection': 1},
 {'_time': 36,
  '_lineIndex': 2,
  '_lineLayer': 0,
  '_type': 1,
  '_cutDirection': 0},
 {'_time': 42,
  '_lineIndex': 2,
  '_lineLayer': 0,
  '_type': 1,
  '_cutDirection': 1},
 {'_time': 43,
  '_lineIndex': 0,
  '_lineLayer': 0,
  '_type': 0,
  '_cutDirection': 1},
 {'_time': 44,
  '_lineIndex': 2,
  '_lineLayer': 0,
  '_type': 1,
  '_cutDirection': 0},
 {'_time': 46,
  '_lineIndex': 1,
  '_lineLayer': 0,
  '_type': 0,
  '_cutDirection': 0},
 {'_time':

In [35]:
df = pd.DataFrame(data['_notes'])
df

,_cutDirection,_lineIndex,_lineLayer,_time,_type
0,1,3,0,12.0,1
1,0,2,0,19.0,1
2,1,2,0,20.0,1
3,1,1,0,27.0,0
4,0,1,0,28.0,0
5,1,2,0,35.0,1
6,0,2,0,36.0,1
7,1,2,0,42.0,1
8,1,0,0,43.0,0
9,0,2,0,44.0,1


In [36]:
(df.apply(lambda x: int(100 * (x['_time'] - math.floor(x['_time']))), axis=1) / 100).value_counts()


0.0    242
0.5      5
dtype: int64

- Most of the blocks appear precisely on the beats.
- Some are on half beats.
- From Expert and harder there are even triolas and wierd stuff

In [37]:
arr = np.zeros(output_shape)
arr[2][3][1][8]

0.0

In [38]:
df['_time'] = round(df['_time'], 2)

def create_ndarr(data):
    arr = np.zeros(output_shape)
    
    for index, row in data.iterrows():
        arr[row['_lineLayer'], row['_lineIndex'], row['_type'], row['_cutDirection']] = 1
    return arr

new_df = pd.DataFrame(df.groupby('_time').apply(create_ndarr), columns=['output'])

In [33]:
new_df['prev'] = 0
last_time = 0

for i, row in new_df.iterrows():
    new_df.loc[i, 'prev'] = i - last_time
    last_time = i

new_df['next'] = new_df['prev'].shift(periods=-1).fillna(0)
new_df

,output,prev,next
_time,,,
12.0,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",12.0,7.0
19.0,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",7.0,1.0
20.0,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",1.0,7.0
27.0,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",7.0,1.0
28.0,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",1.0,7.0
35.0,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",7.0,1.0
36.0,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",1.0,6.0
42.0,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",6.0,1.0
43.0,"[[[[0. 1. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",1.0,1.0


In [41]:
from json_loader import json_to_blockmasks

json_to_blockmasks(extreme_plus)

,output,time,prev,next
_time,,,,
2.334146,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",2.334146,11.832627,0.365854
2.700000,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",2.700000,0.365854,0.182927
2.882927,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",2.882927,0.182927,0.182927
3.065854,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",3.065854,0.182927,0.182927
3.248780,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",3.248780,0.182927,0.182927
3.431707,"[[[[1. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",3.431707,0.182927,0.365854
3.797561,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",3.797561,0.365854,0.182927
3.980488,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",3.980488,0.182927,0.182927
4.163415,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",4.163415,0.182927,0.365854
